In [1]:
# This notebook analyzes the efficacy of different clustering algorithms for matching design patterns and design problems.
# Note: Table C2 in Hussain et al 2017 seems to indicate that fuzzy c-means clustering with binary weighting is the most effective combination for GoF patterns,
# but TF-IDF yields the highest f-value for fuzzy c-means (0.73).
# We are aiming for an f-value of 0.7 or above.

!pip install unidecode
!pip install fuzzy-c-means
!pip install --upgrade scikit-learn
!pip install scikit-learn-extra

from fcmeans                          import FCM

# Data Structures
import numpy  as np
import pandas as pd
import json
# Corpus Processing
import re
import nltk
import nltk.corpus
from nltk.tokenize                    import word_tokenize
from nltk.stem                        import WordNetLemmatizer
from nltk                             import SnowballStemmer, PorterStemmer
nltk.download('punkt')

from sklearn.feature_extraction.text  import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing            import normalize, Normalizer
from sklearn.decomposition            import PCA, TruncatedSVD
from sklearn.cluster                  import KMeans, BisectingKMeans, AgglomerativeClustering
from sklearn_extra.cluster            import KMedoids
from sklearn.pipeline                 import make_pipeline

from unidecode                        import unidecode

# K-Means
from sklearn                          import cluster

# Visualization and Analysis
import matplotlib.pyplot  as plt
import matplotlib.cm      as cm
import seaborn            as sns
from sklearn.metrics                  import silhouette_samples, silhouette_score, confusion_matrix, ConfusionMatrixDisplay, f1_score
from wordcloud                        import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 4.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 KB 5.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.3 requires click>=8.0, but you have click 7.1.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/publi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [29]:
def getTrainData(fileName):
  df = pd.read_csv(fileName)
  df = df.drop_duplicates(subset=['name'])

  return df

In [30]:
# Add the design problem to the dataset.
#problemRow = {'name':"bridge design problem 1", 'correct_category':1, 'overview':"Design a system enabling to display on a screen some empty windows (no button, no menu…). A window can have several different styles depending on the platform used. We consider two platforms, XWindow and Presentation Manager. The client code must be written independently and without knowledge of the future execution platform. It is probable that the system evolves in order to display specialized windows by ‘application windows’ (able to manage applications) and ‘iconised windows’ (with an icon)"}
#problemRow = {'name':"state design problem 2", 'correct_category':0,  'overview':"Design a DVD market place work. The DVD marketplace provides DVD to its clients with three categories: children, normal and new. A DVD is new during some weeks, and after change category. The DVD price depends on the category. It is probable that the system evolves in order to take into account the horror category"}
#problemRow = {'name':"mediator design problem 3", 'correct_category':0,  'overview':"Design the communications of a plane approaching an airport. When a plane approaches an airport, it must announce to all the other planes which are around that it intends to land, and await their confirmation before carrying out the operation. It is the control tower of the airport which guarantees the regulation of the air traffic, by making sure that there is no trajectory or destination conflict between several planes. In addition to the class diagram, you must also submit a collaboration (in the form of a diagram of collaboration or a diagram of objects and sequence) that describes the landing of a plane amidst in a context of two demands to land and one wanting to take off"}
#problemRow = {'name':"composite design problem 4", 'correct_category':1, 'overview':"Design a system enabling you to draw a graphic image. A graphic image is composed of lines, rectangles, texts, and images. An image may be composed of other images, lines, rectangles, and texts"}
#problemRow = {'name':"decorator design problem 5", 'correct_category':1, 'overview':"Design a system enabling you to display visual objects on a screen. A visual object can be composed of one or more texts or images. If needed, the system must allow the addition of a vertical scroll bar, a horizontal scroll bar, an edge and a menu to this object. These additions may be accumulated."}
#problemRow = {'name':"chain of responsbility design problem 6", 'correct_category':0, 'overview':"Design a help manager for a Java application. A help manager allows the display of a help message depending on the objects on which a client has clicked. For example, the “?”, sometimes located near the contextual menu of a Windows dialog box, allows the display of the help related to the button or the area where to click. If the button on which one clicks does not contain help, it is the area container which displays its help, and so on. If any object contains help, the help manager displays /“No help available for this area/”. Instantiate your class diagram in a sequence diagram of the example of a printing window. This window (JDialog) consists in an explanatory text (JLabel) and in a container (JPanel). This last contains a /“Print button/” (JButton) and a /“Cancel button/” (JButton). The /“Print button/” contains help /“Launches the impression of the document/”. The /“Cancel button/” the text as well as the window do not contain help. Lastly, the container contains help /“Click on one of the buttons/”. In the sequence diagram, reveal the scenarios: /“The user asks for the help of the Print button/”, /“the user asks for the help of the Cancel button/”, and /“the user asks for the help of the text/”"}
#problemRow = {'name':"command design problem 7", 'correct_category':0, 'overview':"Design a tutorial to learn how to program a calculator. This calculator executes the four basic arithmetic operations. The goal of this tutorial is to make it possible to take a set of operations to be executed sequentially. The tutorial presents a button for each arithmetic operation, and two input fields for the operands. After each click on a button of an operation, the user has then the choice to start again or execute the sequence of operations to obtain the result. It is probable that this tutorial evolves in order to make it possible for the user to remove the last operation of the list and to take into account the operation of modulo"}
#problemRow = {'name':"visitor design problem 8", 'correct_category':0, 'overview':"Many distinct and unrelated operations need to be performed on node objects in a heterogeneous aggregate structure. You want to avoid ‘polluting’ the node classes with these operations. And, you don't want to have to query the type of each node and cast the pointer to the correct type before performing the desired operation."}
#problemRow = {'name':"adapter design problem 9", 'correct_category':1, 'overview':"Design a drawing editor. A design is composed of te graphics (lines, rectangles and roses), positioned at precise positions. Each graphic form must be modeled by a class that provides a method draw(): void. A rose is a complex graphic designed by a black-box class component. This component performs this drawing in memory, and provides access through a method getRose(): int that returns the address of the drawing. It is probable that the system evolves in order to draw circles"}
# TODO: we need some design problems related to creational patterns
#df = df.append(problemRow, ignore_index=True)


In [31]:
# removes a list of words (ie. stopwords) from a tokenized list.
def removeWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if token not in listOfWords]

# applies stemming to a list of tokenized words
def applyStemming(listOfTokens, stemmer):
    return [stemmer.stem(token) for token in listOfTokens]

# removes any words composed of less than 2 or more than 21 letters
def twoLetters(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

In [32]:
nltk.download('stopwords')
def processCorpus(corpus, language, stemmer):   
    stopwords = nltk.corpus.stopwords.words(language)
    param_stemmer = stemmer
    
    for document in corpus:
        index = corpus.index(document)
        corpus[index] = str(corpus[index]).replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
        corpus[index] = corpus[index].replace(',', '')          # Removes commas
        corpus[index] = corpus[index].rstrip('\n')              # Removes line breaks
        corpus[index] = corpus[index].casefold()                # Makes all letters lowercase
        
        corpus[index] = re.sub('\W_',' ', corpus[index])        # removes specials characters and leaves only words
        corpus[index] = re.sub("\S*\d\S*"," ", corpus[index])   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
        corpus[index] = re.sub("\S*@\S*\s?"," ", corpus[index]) # removes emails and mentions (words with @)
        corpus[index] = re.sub(r'http\S+', '', corpus[index])   # removes URLs with http
        corpus[index] = re.sub(r'www\S+', '', corpus[index])    # removes URLs with www

        listOfTokens = word_tokenize(corpus[index])
        twoLetterWord = twoLetters(listOfTokens)

        listOfTokens = removeWords(listOfTokens, stopwords)
        listOfTokens = removeWords(listOfTokens, twoLetterWord)
        
        listOfTokens = applyStemming(listOfTokens, param_stemmer)

        corpus[index]   = " ".join(listOfTokens)
        corpus[index] = unidecode(corpus[index])

    return corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
# TODO: use k-means before this chunk of code to classify the problem with a pattern class, then perform cosine similarity with the problem and the list of candidate patterns from that class.
# Source: https://danielcaraway.github.io/html/sklearn_cosine_similarity.html

def cosine_sim(df, df_col, class_no, pos_to_last):
  unigram_count = CountVectorizer(encoding='latin-1', binary=False)
  unigram_count_stop_remove = CountVectorizer(encoding='latin-1', binary=False, stop_words='english')

  # get the list of candidate patterns
  txts = df_col.loc[df['Kmeans'] == class_no] # where label == class_no
  vecs = unigram_count.fit_transform(txts)
  
  cos_sim = cosine_similarity(vecs[-pos_to_last], vecs)
  #sim_sorted_doc_idx = cos_sim.argsort()
  # print the most similar pattern to the problem; it's actually the problem itself
  #print("Design Problem: \n" + txts.iloc[sim_sorted_doc_idx[-1][len(txts)-1]] + "\n")

  #bestFittingPatternDesc = txts.iloc[sim_sorted_doc_idx[-1][len(txts)-2]]

  # print the second most similar pattern; it's likely the best-fitting design pattern for the design problem
  #print(txts[sim_sorted_doc_idx[-1][len(txts)-2]])
  #print("\nCorrect Pattern: " + (df['name'][(df['overview'] == bestFittingPatternDesc)]).to_string(index=False) + "\n")

  return cos_sim, txts


In [34]:
def displayPredictions(cos_sim, txts, df):
  sim_sorted_doc_idx = cos_sim.argsort()
  for i in range(len(txts) - 1):
    patternDesc = txts.iloc[sim_sorted_doc_idx[-1][len(txts)-(i + 2)]]
    patternName = (df['name'][(df['overview'] == patternDesc)]).to_string(index=False)
    percentMatch = int((cos_sim[0][sim_sorted_doc_idx[-1][len(txts)-(i + 2)]]) * 100)
    print("{}th pattern:  {:<20}{}%  match".format(i+1, patternName, percentMatch))


In [35]:
def runAlgorithms(num_clusters, final_df, df):
  # bisecting_strategy{“biggest_inertia”, “largest_cluster”}, default=”biggest_inertia”
  final_df_array = final_df.to_numpy()

  Bi_Bisect = BisectingKMeans(n_clusters=num_clusters, bisecting_strategy="biggest_inertia")
  Lc_Bisect = BisectingKMeans(n_clusters=num_clusters, bisecting_strategy="largest_cluster")
  Hierarchy = AgglomerativeClustering(n_clusters=num_clusters)
  Fuzzy_Means = FCM(n_clusters=num_clusters)
  Fuzzy_Means.fit(final_df_array)
  kmed = KMedoids(n_clusters=num_clusters)
  kmed_manhattan = KMedoids(n_clusters=num_clusters,metric='manhattan')
  Kmeans = cluster.KMeans(n_clusters = num_clusters)

  Kmeans_labels = Kmeans.fit_predict(final_df)
  fuzzy_labels = Fuzzy_Means.predict(final_df_array)
  bi_bisect_labels = Bi_Bisect.fit_predict(final_df)
  lc_bisect_labels = Lc_Bisect.fit_predict(final_df)  
  hierarchy_labels = Hierarchy.fit_predict(final_df)
  kmed_labels = kmed.fit_predict(final_df)
  kmed_man_labels = kmed_manhattan.fit_predict(final_df)

  df['Kmeans'] = Kmeans_labels
  df['fuzzy'] = fuzzy_labels
  df['hierarchy'] = hierarchy_labels
  df['Bi_Bisect'] = bi_bisect_labels  
  df['Lc_Bisect'] = lc_bisect_labels
  df['PAM-EUCLIDEAN'] = kmed_labels
  df['PAM-MANHATTAN'] = kmed_man_labels


In [36]:
def validateInput(designProblem):
  numOfWords = len(designProblem.split())
  if (numOfWords < 30 or numOfWords > 120):
    return False
  return True

def main():
  language = 'english'
  stemmer = PorterStemmer()
  vectorizer = TfidfVectorizer(sublinear_tf=True)
  df = getTrainData("GOF Patterns (2.0).csv")

  while(True):
    # designProblem = input("Enter your design problem: \n")

    # if(not validateInput(designProblem)):
    #   print("Invalid input size! please try again. \n")
    #   continue

    # problemRow = {'name':"design problem", 'correct_category':4, 'overview': designProblem}
    # df = df.append(problemRow, ignore_index=True)

    corpus = df['overview'].tolist()
    corpus = processCorpus(corpus, language, stemmer)

    X = vectorizer.fit_transform(corpus)
    tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names_out())

    runAlgorithms(3, tf_idf, df)
    cos_sim, txts = cosine_sim(df, df['overview'], df['Kmeans'].iloc[df.index[-1]], 1)
    #displayPredictions(cos_sim, txts, df)
    return df, cos_sim, txts, tf_idf

df, cos_sim, txts, tfidf = main()
df

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,Unnamed: 0,name,correct_category,overview,Kmeans,fuzzy,hierarchy,Bi_Bisect,Lc_Bisect,PAM-EUCLIDEAN,PAM-MANHATTAN
0,0,abstract factory,2,Provide an interface for creating families of ...,2,2,0,1,2,2,1
1,1,builder,2,The Builder pattern captures all these relatio...,0,2,0,1,2,1,0
2,2,factory method,2,"Define an interface for creating an object, bu...",2,1,0,0,1,2,2
3,3,prototype,2,Specify the kinds of objects to create using a...,2,1,2,1,0,0,0
4,4,singleton,2,"Ensure a class only has one instance, and prov...",2,0,2,0,0,0,0
5,5,adapter,1,Convert the interface of a class into another ...,2,2,0,2,0,2,1
6,6,bridge,1,Decouple an abstraction from its implementatio...,2,1,0,1,2,2,1
7,7,composite,1,Compose objects into tree structures to repres...,1,0,1,1,2,1,0
8,8,decorator,1,Attach additional responsibilities to an objec...,2,0,0,1,0,0,1
9,9,facade,1,Provide a unified interface to a set of interf...,2,0,0,2,0,0,1


In [37]:
def Silhouette(vector_data, cluster_labels):
  # range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9]
  #silhouette_avg = []
  # for num_clusters in range_n_clusters:
  #   # initialise kmeans
  #   kmeans = KMeans(n_clusters=num_clusters, n_init='auto')
  #   kmeans.fit(vector_data)
  #   cluster_labels = kmeans.labels_
    
    # silhouette score
    #silhouette_avg.append(silhouette_score(vector_data, cluster_labels))
  s_avg = silhouette_score(vector_data, cluster_labels)
  return s_avg
  # plt.plot(range_n_clusters,silhouette_avg,'bx-')

  # plt.xlabel('Values of K') 
  # plt.ylabel('Silhouette score') 
  # plt.title('Silhouette analysis For Optimal k')
  # plt.show()

In [38]:
def getFScore(labels, df):
  df2 = df.pivot_table(index = ['correct_category'], aggfunc ='size')

  num_of_creational = df2[2]
  num_of_structural = df2[1]
  num_of_behavioral = df2[0]

  true_1 = [0]*num_of_creational + [1]*num_of_structural + [2]*num_of_behavioral
  true_2 = [0]*num_of_creational + [2]*num_of_structural + [1]*num_of_behavioral
  true_3 = [1]*num_of_creational + [0]*num_of_structural + [2]*num_of_behavioral
  true_4 = [1]*num_of_creational + [2]*num_of_structural + [0]*num_of_behavioral
  true_5 = [2]*num_of_creational + [0]*num_of_structural + [1]*num_of_behavioral
  true_6 = [2]*num_of_creational + [1]*num_of_structural + [0]*num_of_behavioral

  #print('===========KMEANS===========')
  #print('Predicted labels:')
  #display(Kmeans_labels.tolist())

  fscores = [
      f1_score(true_1, labels.tolist(),average='micro'),
      f1_score(true_2, labels.tolist(),average='micro'),
      f1_score(true_3, labels.tolist(),average='micro'),
      f1_score(true_4, labels.tolist(),average='micro'),
      f1_score(true_5, labels.tolist(),average='micro'),
      f1_score(true_6, labels.tolist(),average='micro')
  ]

  km_best = np.around(max(fscores),3)
  #print('\nBest fscore is:', km_best, 'from true_' + str(np.argmax(fscores) + 1))
  #display(globals()['true_' + str(np.argmax(fscores) + 1)])
  return km_best

In [58]:
np.random.seed(8)

kmeans_silhouette_avg = []
kmeans_fscore_avg = []

fmeans_silhouette_avg = []
fmeans_fscore_avg = []

kmed_silhouette_avg = []
kmed_fscore_avg = []

num_clusters = 3
final_df = tfidf
final_df_array = final_df.to_numpy()

#Bi_Bisect = BisectingKMeans(n_clusters=num_clusters, bisecting_strategy="biggest_inertia")
#Lc_Bisect = BisectingKMeans(n_clusters=num_clusters, bisecting_strategy="largest_cluster")
#Hierarchy = AgglomerativeClustering(n_clusters=num_clusters)
Fuzzy_Means = FCM(n_clusters=num_clusters)

kmed = KMedoids(n_clusters=num_clusters)
#kmed_manhattan = KMedoids(n_clusters=num_clusters,metric='manhattan')
Kmeans = cluster.KMeans(n_clusters = num_clusters, n_init='auto', random_state=999)

for i in range(20):
    Fuzzy_Means.fit(final_df_array)
    Kmeans_labels = Kmeans.fit_predict(final_df)
    fuzzy_labels = Fuzzy_Means.predict(final_df_array)
    #bi_bisect_labels = Bi_Bisect.fit_predict(final_df)
    #lc_bisect_labels = Lc_Bisect.fit_predict(final_df)  
    #hierarchy_labels = Hierarchy.fit_predict(final_df)
    kmed_labels = kmed.fit_predict(final_df)
    #kmed_man_labels = kmed_manhattan.fit_predict(final_df)

    # # initialise kmeans
    # kmeans = KMeans(n_clusters=3, n_init="auto")
    # kmeans.fit(tfidf)

    kmeans_silhouette_avg.append(getFScore(Kmeans_labels, df))
    kmeans_fscore_avg.append(Silhouette(tfidf, Kmeans_labels))

    fmeans_silhouette_avg.append(getFScore(fuzzy_labels, df))
    fmeans_fscore_avg.append(Silhouette(tfidf, fuzzy_labels))

    kmed_silhouette_avg.append(getFScore(kmed_labels, df))
    kmed_fscore_avg.append(Silhouette(tfidf, kmed_labels))

    print(Kmeans_labels)

print("\n===========kmeans===========\n")
print(sum(kmeans_fscore_avg) / len(kmeans_fscore_avg))
print(sum(kmeans_silhouette_avg) / len(kmeans_silhouette_avg))

print("\n===========fmeans===========\n")
print(sum(fmeans_fscore_avg) / len(fmeans_fscore_avg))
print(sum(fmeans_silhouette_avg) / len(fmeans_silhouette_avg))

print("\n============kmed============\n")
print(sum(kmed_fscore_avg) / len(kmed_fscore_avg))
print(sum(kmed_silhouette_avg) / len(kmed_silhouette_avg))

[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]
[0 0 0 0 2 0 0 1 2 2 0 2 1 1 1 1 1 2 0 1 1 1 1]

===========kmeans===========

0.0079851